# Post-process vasca_pipe results
Make catalog file of selected sources only, associate and get store additional info for listed sources

In [1]:
region_name = "ALL_10-800" #"CAINGSGII_10-800"  #"TDS" #"CAINGSGII_10-800"  #"TDS" # "MDIS_10-800" # "TDS" # _ELAISN1
region_fname = "./vasca_pipeline/"+region_name+"/region_"+region_name+".fits"
region_cat_fname = region_fname.replace(".fits","_cat.fits")

do_cat = False                        # Recreate catalog file from vasca_pipe output region
do_cross_match = False            # Does source matching with CDS, typically for all (~10k) catalog sources
do_variable_addition = True        # Adds variables of matched sources to tt_sources
do_LombScargle = False             # Run LombScargle, typically for all (~10k) catalog sources
do_src_prep = False                 # Gets additional info (e.g. SED from Vizer) for sources listes in srcs_ids list below. This is slow, and therefore only for the sources listed in srcs_ids below

srcs_ids = [535864 ,451644 ,541266 ,581995 ,187856 ,166179 ,34658 ,426363 ,151796 ,259271 ,419001 ,25273 ,]

In [2]:
%matplotlib widget
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np

from astropy.coordinates import SkyCoord
from astropy.table import unique, Table, hstack, join, setdiff
from astropy import units as uu
from astroquery.simbad import Simbad
from astropy.visualization import quantity_support
from astropy.modeling.models import BlackBody
from astropy import constants as cc
from matplotlib.ticker import ScalarFormatter
from astroquery.sdss import SDSS

from vasca.region import Region
from vasca.field import BaseField
import vasca.visualization as vvis
from vasca.tables import TableCollection, dd_vasca_columns
from vasca.utils import otype2ogroup,dd_ogrp2otypes, dd_ogrp2col, dd_filter2wavelength, mag2flux, flux2mag, query_vizier_sed, add_ogrp

from loguru import logger
logger.enable("vasca")

# Do catalog file

if do_cat:
    rg = Region()
    rg.load_from_fits(region_fname) 
    rc = rg.get_region_catalog()
else:
    rc = Region()
    rc.load_from_fits(region_cat_fname)

#Cross match to SIMBAD and GAIA
if do_cross_match:
    rc.cross_match_cds(query_table="J/MNRAS/508/3877/maincat")
    
    rc.cross_match_cds(query_table="simbad", overwrite=False)
    rc.cross_match_cds(overwrite=True)

    #Crossmatch with GFCAT
    tt_cat = Table.read("./resources/GFCAT/hlsp_gfcat_galex_imaging_object-cat_fuv-nuv_v1_table.ecsv")
    rc.add_table(tt_cat,"tt_gfcat")
    rc.cross_match(dist_max=1.5 * uu.arcsec, dist_s2n_max=None, cat_table_name="tt_gfcat",cat_id_name="gfcat_src_id",cat_name="gfcat", src_table_name="tt_sources")

    #rc.cross_match_cds(query_table="I/358/varisum",vizier_columns=["*", "NG", "dGmag"],)

#Add variables
if do_variable_addition:
    rc.add_column("tt_gaiadr3", "Gmag_abs")
    sel_plx = rc.tt_gaiadr3["RPlx"]>4
    rc.tt_gaiadr3["Gmag_abs"][sel_plx]=rc.tt_gaiadr3["Gmag"][sel_plx] + 5.0 * np.log10(rc.tt_gaiadr3["Plx"][sel_plx]) - 10.0
    rc.tt_gaiadr3["Plx_dist"] = rc.tt_gaiadr3["Plx"].quantity.to(uu.parsec, equivalencies=uu.parallax())

    # Add normalized excess
    rc.tt_sources["flux_ne"]=np.sqrt(rc.tt_sources["flux_nxv"])

    # Add object groups
    add_ogrp(rc.tt_simbad)
    add_ogrp(rc.tt_gaiadr3,provenance="GAIA")

    src_coords = SkyCoord(ra=rc.tt_sources["ra"].quantity, dec=rc.tt_sources["dec"].quantity, frame='icrs')
    #print(src_coords.to_string('hmsdms')) #'decimal'
    src_names = [] 
    for coord in src_coords:
        scoord = coord.to_string('hmsdms', precision=1, alwayssign=True, pad=False, fields=3, format=None)
        scoord = scoord.replace("s ","").replace("s","").replace("d","").replace("m","").replace("h","")
        src_names.append(("1UVA J"+scoord).replace("J+","J"))
    rc.add_column("tt_sources", "src_name", col_data=src_names)

if do_LombScargle:
    rc.set_LombScargle(nbins_min=20)

#Write out all changes to region
rc.write_to_fits(region_cat_fname, overwrite=True)

#Prepare sources, including Vizier SED.
if do_src_prep:
    for src_id in srcs_ids:
        print(f"*********** Creating Source Table ID {src_id} *****************")
        tc_src = rc.get_src_from_id(src_id, load_from_file = False)
print("Done")

2024-01-11 10:33:59.426 | DEBUG    | vasca.tables:load_from_fits:328 - Loading file with name './vasca_pipeline/ALL_10-800/region_ALL_10-800_cat.fits'
2024-01-11 10:33:59.436 | DEBUG    | vasca.tables:load_from_fits:339 - Loading table 'tt_fields'
2024-01-11 10:33:59.451 | DEBUG    | vasca.tables:load_from_fits:339 - Loading table 'tt_filters'
2024-01-11 10:33:59.468 | DEBUG    | vasca.tables:load_from_fits:339 - Loading table 'tt_visits'
2024-01-11 10:33:59.492 | DEBUG    | vasca.tables:load_from_fits:339 - Loading table 'tt_sources'
2024-01-11 10:33:59.555 | DEBUG    | vasca.tables:load_from_fits:339 - Loading table 'tt_detections'
2024-01-11 10:33:59.608 | DEBUG    | vasca.tables:load_from_fits:339 - Loading table 'tt_coadd_detections'
2024-01-11 10:33:59.733 | DEBUG    | vasca.tables:load_from_fits:339 - Loading table 'tt_coadd_sources'
2024-01-11 10:33:59.776 | DEBUG    | vasca.tables:load_from_fits:339 - Loading table 'tt_src_id_map'
2024-01-11 10:33:59.793 | DEBUG    | vasca.tab

Done
